
# Geração de sensibilidades de mercado no sistema.dat


Para realizar a análise do sistema.dat, será utilizado o módulo plotly



In [8]:
from datetime import datetime
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio

pio.templates.default = "ggplot2"

O sistema.dat é o arquivo de entrada do modelo NEWAVE que contém informações
sobre os submercados de energia. Em particular, são definidos os submercados
e as curvas de mercado de energia esperado por estágio, para cada um.
É uma análise comum a análise de sensibilidades em relação ao mercado de energia
e à geração das usinas não simuladas e, por isso, será ilustrado um exemplo deste caso.



In [9]:
from inewave.newave import Sistema

arq_sistema = Sistema.read("./deck_newave_2024_08_inewave/sistema.dat")

A definição dos submercados é acessível através do bloco que define os custos de déficit



In [10]:
arq_sistema.custo_deficit

,codigo_submercado,nome_submercado,ficticio,patamar_deficit,custo,corte
0,1,SUDESTE,0,1,7810.62,1.0
1,1,SUDESTE,0,2,0.00,0.0
2,1,SUDESTE,0,3,0.00,0.0
3,1,SUDESTE,0,4,0.00,0.0
4,2,SUL,0,1,7810.62,1.0
5,2,SUL,0,2,0.00,0.0
6,2,SUL,0,3,0.00,0.0
7,2,SUL,0,4,0.00,0.0
8,3,NORDESTE,0,1,7810.62,1.0
9,3,NORDESTE,0,2,0.00,0.0


As informações de mercado de energia são reunidas em uma única propriedade
e esta pode ser alterada livremente



In [11]:
arq_sistema.mercado_energia

,codigo_submercado,data,valor
0,1,2024-01-01 00:00:00,NaN
1,1,2024-02-01 00:00:00,NaN
2,1,2024-03-01 00:00:00,NaN
3,1,2024-04-01 00:00:00,NaN
4,1,2024-05-01 00:00:00,NaN
...,...,...,...
283,4,9999-08-01 00:00:00,8682.0
284,4,9999-09-01 00:00:00,8777.0
285,4,9999-10-01 00:00:00,8784.0
286,4,9999-11-01 00:00:00,8528.0


Será feito um gráfico de área empilhado. Para isso, serão geradas algumas variáveis auxiliares.



In [12]:
df = arq_sistema.mercado_energia
x = pd.date_range(
    df["data"].min(),
    df["data"].max(),
    freq="MS",
)

OutOfBoundsDatetime: Cannot cast 9999-12-01 00:00:00 to unit='ns' without overflow.

Para a figura, são geradas as retas independentemente



In [ ]:
fig = go.Figure()
for submercado in df["codigo_submercado"].unique():
    y = df.loc[df["codigo_submercado"] == submercado, "valor"]
    fig.add_trace(
        go.Scatter(
            x=x,
            y=y,
            mode="lines",
            stackgroup="one",
            name=str(submercado),
        )
    )

fig.update_xaxes(title="Data")
fig.update_yaxes(title="Mercado de Energia (MWmes)")
fig.update_layout(legend_title_text="Submercado")
fig

É possível realizar edições livres na propriedade do arquivo, para geração de estudos
de sensibilidades. Por exemplo, é possível aumentar a carga do submercado NORDESTE
em 30% e conferir o efeito na operação com a execução do modelo.



In [ ]:
arq_sistema.mercado_energia.loc[
    arq_sistema.mercado_energia["codigo_submercado"] == 3, "valor"
] *= 1.3

from io import StringIO

conteudo_sistema = StringIO()
arq_sistema.write(conteudo_sistema)
print(conteudo_sistema.getvalue())